In [1]:
import numpy as np
import scipy as sp

from scipy.stats import norm

In [2]:
S0 = 50
T = 0.5
K = 50
r = 0.1
q = 0.05
sigma = 0.4
n = 10000

## Black-Scholes

$$ c(S_0, 0) = S_0N(d_1) - Ke^{-rT}N(d_2)$$
$$d1 = \frac{\ln(\frac{S_0}{K}) + (r-q+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}} \quad\quad d2 = d1 - \sigma\sqrt{T}$$

In [21]:
d1 = np.log(S0 / K) + ( r - q + (sigma**2) / 2) * T
d1 = d1 / (sigma* np.sqrt(T))
d2 = d1 - (sigma* np.sqrt(T))

print("-->", np.exp(-q*T)* S0 * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2))

--> 6.039620873020631


## Monte Carlo

In [22]:

avg = []
tmp = []
for i in range(n_rep):
    payoff = np.random.normal(np.log(S0) + (r - q - (sigma**2) / 2) * T, (sigma) * np.sqrt(T), n_sim)
    payoff = np.exp(payoff)
    for num in payoff:
        if num >= K:
            tmp.append(num - K)
        else:
            tmp.append(0)
    avg.append(np.exp(-r * T) * np.mean(tmp))
    print(avg[-1])

6.077277987735847
5.98119763921856
5.987889709209967
5.996685237951905
6.0466397047366
6.065132398435066
6.055447309844451
6.060035121327157
6.052907571607865
6.055737683035963
6.060576295922217
6.072197234578118
6.0731243060485385
6.0676480219194495
6.062755297100838
6.057024777116246
6.05513509666765
6.059611966733679
6.0653045973036965
6.05880684794991


In [23]:
np.mean(avg) - 2 * np.std(avg), np.mean(avg) + 2 * np.std(avg)

(5.996377046256765, 6.104736434187607)

## CRR Binomial Tree

In [3]:
delta_t = T/n
u = np.exp(sigma * np.sqrt(delta_t))
d = 1 / u
p = (np.exp((r-q)*delta_t) - d)/(u-d)
print("delta t:{}\nu:{}\t d:{}\np:{}".format(delta_t, u, d, p))

delta t:5e-05
u:1.0028324308986505	 d:0.9971755691066828
p:0.49973483539162694


### European option

In [15]:
cnj = []
n = 500

In [16]:
crr_tree = np.zeros((n+1, n+1))
crr_tree.shape

(501, 501)

In [17]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            crr_tree[row, col] =max(0, stock_value - K)
    else:
#         print(row, col)
        for row in range(0, col+1):
#             print(crr_tree[row, col], crr_tree[row, col + 1], crr_tree[row + 1, col + 1])
            crr_tree[row, col] = np.exp(-r*delta_t)* (p * crr_tree[row, col + 1] + (1 - p) * crr_tree[row + 1, col + 1])

500 499 498 497 496 495 494 493 492 491 490 489 488 487 486 485 484 483 482 481 480 479 478 477 476 475 474 473 472 471 470 469 468 467 466 465 464 463 462 461 460 459 458 457 456 455 454 453 452 451 450 449 448 447 446 445 444 443 442 441 440 439 438 437 436 435 434 433 432 431 430 429 428 427 426 425 424 423 422 421 420 419 418 417 416 415 414 413 412 411 410 409 408 407 406 405 404 403 402 401 400 399 398 397 396 395 394 393 392 391 390 389 388 387 386 385 384 383 382 381 380 379 378 377 376 375 374 373 372 371 370 369 368 367 366 365 364 363 362 361 360 359 358 357 356 355 354 353 352 351 350 349 348 347 346 345 344 343 342 341 340 339 338 337 336 335 334 333 332 331 330 329 328 327 326 325 324 323 322 321 320 319 318 317 316 315 314 313 312 311 310 309 308 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 

In [18]:
crr_tree[0,0]

1.289801259428918

In [19]:
del crr_tree

### American option

In [20]:
cnj = []
n = 500

In [21]:
crr_tree = np.zeros((n+1, n+1))
crr_tree.shape

(501, 501)

In [22]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            crr_tree[row, col] =max(0, stock_value - K)
    else:
#         print(row, col)
        for row in range(0, col+1):
            stock_value = S0 * (u ** (col-row)) * (d ** row)
            crr_tree[row, col] = np.exp(-r*delta_t)* (p * crr_tree[row, col + 1] + (1 - p) * crr_tree[row + 1, col + 1])
            crr_tree[row, col] = max(crr_tree[row, col], stock_value - K)

500 499 498 497 496 495 494 493 492 491 490 489 488 487 486 485 484 483 482 481 480 479 478 477 476 475 474 473 472 471 470 469 468 467 466 465 464 463 462 461 460 459 458 457 456 455 454 453 452 451 450 449 448 447 446 445 444 443 442 441 440 439 438 437 436 435 434 433 432 431 430 429 428 427 426 425 424 423 422 421 420 419 418 417 416 415 414 413 412 411 410 409 408 407 406 405 404 403 402 401 400 399 398 397 396 395 394 393 392 391 390 389 388 387 386 385 384 383 382 381 380 379 378 377 376 375 374 373 372 371 370 369 368 367 366 365 364 363 362 361 360 359 358 357 356 355 354 353 352 351 350 349 348 347 346 345 344 343 342 341 340 339 338 337 336 335 334 333 332 331 330 329 328 327 326 325 324 323 322 321 320 319 318 317 316 315 314 313 312 311 310 309 308 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 

In [23]:
crr_tree[0,0]

1.289801259428918

## Bonus

### European

In [24]:
cnj = []
n = 500

In [25]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            cnj.append(max(0, stock_value - K))
    else:
#         print(row, col)
        for row in range(0, col+1):
#             print(crr_tree[row, col], crr_tree[row, col + 1], crr_tree[row + 1, col + 1])
            cnj[row] = np.exp(-r*delta_t)* (p * cnj[row] + (1 - p) * cnj[row + 1])

500 499 498 497 496 495 494 493 492 491 490 489 488 487 486 485 484 483 482 481 480 479 478 477 476 475 474 473 472 471 470 469 468 467 466 465 464 463 462 461 460 459 458 457 456 455 454 453 452 451 450 449 448 447 446 445 444 443 442 441 440 439 438 437 436 435 434 433 432 431 430 429 428 427 426 425 424 423 422 421 420 419 418 417 416 415 414 413 412 411 410 409 408 407 406 405 404 403 402 401 400 399 398 397 396 395 394 393 392 391 390 389 388 387 386 385 384 383 382 381 380 379 378 377 376 375 374 373 372 371 370 369 368 367 366 365 364 363 362 361 360 359 358 357 356 355 354 353 352 351 350 349 348 347 346 345 344 343 342 341 340 339 338 337 336 335 334 333 332 331 330 329 328 327 326 325 324 323 322 321 320 319 318 317 316 315 314 313 312 311 310 309 308 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 

In [26]:
cnj[0]

1.289801259428918

### American

In [27]:
cnj = []

In [28]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            cnj.append(max(0, stock_value - K))
    else:
#         print(row, col)
        for row in range(0, col+1):
            stock_value = S0 * (u ** (col-row)) * (d ** row)
            cnj[row] = np.exp(-r*delta_t)* (p * cnj[row] + (1 - p) * cnj[row + 1])
            cnj[row] = max(cnj[row], stock_value - K)

500 499 498 497 496 495 494 493 492 491 490 489 488 487 486 485 484 483 482 481 480 479 478 477 476 475 474 473 472 471 470 469 468 467 466 465 464 463 462 461 460 459 458 457 456 455 454 453 452 451 450 449 448 447 446 445 444 443 442 441 440 439 438 437 436 435 434 433 432 431 430 429 428 427 426 425 424 423 422 421 420 419 418 417 416 415 414 413 412 411 410 409 408 407 406 405 404 403 402 401 400 399 398 397 396 395 394 393 392 391 390 389 388 387 386 385 384 383 382 381 380 379 378 377 376 375 374 373 372 371 370 369 368 367 366 365 364 363 362 361 360 359 358 357 356 355 354 353 352 351 350 349 348 347 346 345 344 343 342 341 340 339 338 337 336 335 334 333 332 331 330 329 328 327 326 325 324 323 322 321 320 319 318 317 316 315 314 313 312 311 310 309 308 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 

In [29]:
cnj[0]

1.289801259428918

## Combinatorial

In [4]:
import scipy as sp
from scipy.special import comb

In [8]:
print(S0)

50


In [21]:
def mycomb(n, j):
    total = 0
    cnt = 1
    for i in range(n, n-j, -1):
        total += np.log(i)
        total -= np.log(cnt)
        cnt += 1
    return total

In [24]:
np.log(comb(10000,100, exact=True))

TypeError: loop of ufunc does not support argument 0 of type int which has no callable log method

In [22]:
n = 10000
total = 0
for j in range(0, n):
#     print(j)
    total += np.exp(np.log(comb(n, j, exact=True)) + (n-j)* np.log(p) + j * np.log((1-p)) + np.log(max(S0*(u ** (n - j)) * d**j - K, 0)))
#     total += np.exp(mycomb(n, j) + (n-j)* np.log(p) + j * np.log(1-p) + np.log(max(S0*(u ** (n - j)) * d**j - K, 0 + 0.000000000001 )))
    
    if j % 100 == 0:
        print(j)
total * np.exp(-r*T)

0


TypeError: loop of ufunc does not support argument 0 of type int which has no callable log method

In [20]:
n = 10000
total = 0
for j in range(0, n):
#     print(j)
    total += np.exp(np.log(comb(n, j)) + (n-j)* np.log(p) + j * np.log((1-p)) + np.log(max(S0*(u ** (n - j)) * d**j - K, 0)))
#     total += np.exp(mycomb(n, j) + (n-j)* np.log(p) + j * np.log(1-p) + np.log(max(S0*(u ** (n - j)) * d**j - K, 0 + 0.000000000001 )))
    
    if j % 100 == 0:
        print(j)
total * np.exp(-r*T)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


6.039484954833866

In [373]:
def large_fac(number):
    largest_num = 100000
    result = [0] * largest_num
    result[0] = 1
    for i in range(1, number+1):
        prod = 0
        for j in range(len(result)):
            prod = result[j] * i + prod
            result[j] = prod % 10
            prod = prod // 10
    result.reverse()
    for i in range(largest_num):
        if result[i] != 0:
            for j in range(i, largest_num):
                print(result[j], end='')
            break

In [374]:
large_fac(1000)

4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

In [308]:
np.exp()

5.653408585997611e+21

In [310]:
n = 10; j = 3

np.arange(n, n-j, -1)

array([10,  9,  8])